In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_percentage_error
from deap import base, creator, tools, algorithms

In [2]:
data = pd.read_csv('DATA_KOTA.csv')

df = pd.DataFrame(data)

# Normalisasi MinMax
scaler = MinMaxScaler()
df['jumlah_kecelakaan'] = scaler.fit_transform(df[['jumlah_kecelakaan']])

# Menambahkan kolom lag (pergeseran)
df['jumlah_kecelakaan_lag'] = df['jumlah_kecelakaan'].shift(1)
df = df.dropna() # Menghapus baris dengan nilai NaN

In [3]:
# Memisahkan data latih dan data uji sesuai rasio (0.8 untuk 80:20)
train_size = int(0.8 * len(df))
train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]

# Memisahkan fitur dan target untuk data latih
X_train = train_data['jumlah_kecelakaan_lag'].values.reshape(-1, 1)
y_train = train_data['jumlah_kecelakaan'].values

# Memisahkan fitur dan target untuk data uji
X_test = test_data['jumlah_kecelakaan_lag'].values.reshape(-1, 1)
y_test = test_data['jumlah_kecelakaan'].values

In [4]:
# Define evaluation function
def evaluate(individual):
    C, epsilon, gamma = individual
    if C <= 0 or epsilon <= 0 or gamma <= 0:
        return float('inf'),
    svr = SVR(C=C, epsilon=epsilon, gamma=gamma)
    svr.fit(X_train, y_train)
    y_pred = svr.predict(X_test)
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100
    return mape,
    
# Setup DEAP framework
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Parameter ranges
C_range = (10, 50)
epsilon_range = (0.001, 5)
gamma_range = (0.001, 5)

def create_individual():
    C = random.uniform(*C_range)
    epsilon = random.uniform(*epsilon_range)
    gamma = random.uniform(*gamma_range)
    return creator.Individual([C, epsilon, gamma])
    
toolbox.register("individual", create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxUniform, indpb=0.5)  # Menggunakan crossover uniform 
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2) # Menggunakan mutasi gaussian
toolbox.register("select", tools.selTournament, tournsize=3) # Menggunakan seleksi turnamen
toolbox.register("evaluate", evaluate)

In [5]:
# Initialize population
population_size = 300
population = toolbox.population(n=population_size)

# Genetic algorithm parameters    
num_generations = 60
crossover_rate = 0.7
mutation_rate = 0.3

# Run genetic algorithm
for generation in range(num_generations):
    print(f"Generasi {generation + 1}:")
    offspring = algorithms.varAnd(population, toolbox, cxpb=crossover_rate, mutpb=mutation_rate)
    for ind in offspring:
        ind[0] = max(min(ind[0], C_range[1]), C_range[0])
        ind[1] = max(min(ind[1], epsilon_range[1]), epsilon_range[0])
        ind[2] = max(min(ind[2], gamma_range[1]), gamma_range[0])
        
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
        print(f"Individu: {ind}, MAPE: {fit[0]}")
    
    population = toolbox.select(offspring, k=len(population))

# Get the best individual
best_individual = tools.selBest(population, k=1)[0]
best_mape = best_individual.fitness.values[0]
C, epsilon, gamma = best_individual

# Test the best SVR model
best_svr = SVR(kernel='rbf', C=C, epsilon=epsilon, gamma=gamma)
best_svr.fit(X_train, y_train)
y_pred = best_svr.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred) * 100

print("\nParameter terbaik:")
print(f"C: {C}")
print(f"Epsilon: {epsilon}")
print(f"Gamma: {gamma}")
print(f"MAPE: {mape:.4f}")

Generasi 1:
Individu: [27.348141776025884, 3.035102835033554, 1.8144026969638103], MAPE: 38.81718975468974
Individu: [35.768712523113294, 1.4940004879298328, 2.974399034622547], MAPE: 38.81718975468976
Individu: [22.275160938275, 0.5061464667206658, 2.658820015289802], MAPE: 38.81718975468976
Individu: [19.238831802496428, 2.705575325898591, 2.9846416428088487], MAPE: 38.81718975468976
Individu: [25.978702526838283, 3.0961950500381046, 4.538547202686083], MAPE: 38.81718975468974
Individu: [14.61863919146817, 4.972560085546843, 4.611440167896839], MAPE: 38.81718975468977
Individu: [40.1215446132516, 0.3926843993382774, 1.0945255291652622], MAPE: 38.81718975468976
Individu: [33.918163931473345, 0.8160553119993953, 4.821701939872347], MAPE: 38.81718975468976
Individu: [11.16562834708227, 0.8109885361790728, 1.9258052890627693], MAPE: 38.81718975468976
Individu: [16.452265233544697, 1.5980841604924616, 4.745646658725781], MAPE: 38.81718975468976
Individu: [36.29236417541318, 0.701615214723

In [6]:
# Prediksi 6 bulan ke depan
last_value = df['jumlah_kecelakaan'].values[-1] # Nilai terakhir dari data sebenarnya
predicts = []

for i in range(6):
    next_value = best_svr.predict(np.array([[last_value]]))[0]
    predicts.append(next_value)
    last_value = next_value

# Denormalisasi prediksi
predicts = scaler.inverse_transform(np.array(predicts).reshape(-1, 1))

print("Prediksi untuk 6 bulan ke depan:")
print(predicts)

Prediksi untuk 6 bulan ke depan:
[[7.28586016]
 [7.53164193]
 [7.74519495]
 [7.93206483]
 [8.09634823]
 [8.24119355]]
